# Extract FATS for Corot

In [34]:
import os, glob, warnings, FATS, fnmatch
import numpy as np
import pandas as pd
import multiprocessing
from joblib import Parallel, delayed
import time
import matplotlib.pyplot as plt
from matplotlib import pylab 

warnings.filterwarnings("ignore", category=DeprecationWarning) 
%matplotlib inline
pylab.rcParams['figure.figsize'] = 16,9

## Settings
- n_cores = la cantidad de cores que vamos a usar para correrlo en paralelo
- types= Los dos tipos de estrellas de Corot. Solo sirve para poder meterse a los archivos
- n= Representa la cantidad de tiempos a considerar

In [35]:
n_cores = multiprocessing.cpu_count() - 1
folder_path = './Fats/'
feature_space = FATS.FeatureSpace(Data= ['magnitude','time','error'], featureList=None)

base_path = '/Users/Carlos/Desktop/ExpI/'
exp_corot = base_path + 'Corot'     #'*.csv'
exp_ogle = base_path + 'OGLE-III' #'OGLE-*.dat'
exp_vvv = base_path + 'VVV'      #'*.csv'

## Number of Points per Light Curve
n = 500

### Metodos

In [36]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

def get_files(directory, pattern = "*.csv"):
    matches = []
    for root, dirs, files in os.walk(directory):
        for basename in files:
            if fnmatch.fnmatch(basename, pattern):
                filename = os.path.join(root, basename)
                matches.append(filename)
    return matches

def open_corot(path, n, columns):
    df = pd.read_csv(path, comment='#', sep=',')
    df = df[df.DATEBARTT > 0]

    # 3 Desviaciones Standard
    #df = df[np.abs(df.mjd-df.mjd.mean())<=(3*df.mjd.std())]
    
    time = np.array(df[df.columns[columns[0]]].values, dtype=float)
    magnitude = np.array(df[df.columns[columns[1]]].values, dtype=float)
    error = np.array(df[df.columns[columns[2]]].values, dtype=float)
    
    # Not Nan
    not_nan = np.where(~np.logical_or(np.isnan(time), np.isnan(magnitude)))[0]
    time = time[not_nan]
    magnitude = magnitude[not_nan]
    error = error[not_nan]
    
    if len(time) > n:
        time = time[:n]
        magnitude = magnitude[:n]
        error = error[:n]
    
    return time, magnitude, error

def calculate_one_band_features(path, n, columns):
    time, mag, error = open_corot(path, n, columns)
    preproccesed_data = FATS.Preprocess_LC(mag, time, error)
    lc = np.array(preproccesed_data.Preprocess())
    # feature_space = FATS.FeatureSpace(Data= ['magnitude','time','error'], featureList=None)
    feature_space.calculateFeature(lc)
    return  feature_space.result(method='dict')

def save_file(filename, dictionary):    
    new_df = pd.DataFrame(dictionary, index=['0'])
    new_df.to_csv(filename, float_format='%12.6f', sep=",", index=False)

def run_FATS(path, n):
    print '\t ', path
    fats = [-1]*59
    try:
        columns = [0, 1, 2]
        if 'EN2_STAR_CHR' in path:
            columns = [0, 4, 8]

        # Save Fats into folder
        directory = os.path.dirname(path) + '/Fats'
        if not os.path.exists(directory):
            print '[+] Creando Directorio \n\t ->', directory
            os.mkdir(directory)    

        directory += '/' + str(n)
        if not os.path.exists(directory):
            print '[+] Creando Directorio \n\t ->', directory
            os.mkdir(directory)        

        name = os.path.basename(path)[:-4] + '.npy'
        new_path = directory + '/' + name

        # Calculate Fats
        if not os.path.exists(new_path):
            fats = calculate_one_band_features(path, n, columns)
            np.save(new_path, fats)
    except: 
        print '\t\t\t Fatal Error'
    return fats


# Corot 

In [29]:
files = get_files(exp_corot)
print '\t\t [+] Total:', len(files)
batchs = list(chunks(files, 10)) 
for batch in batchs:
    result = Parallel(n_jobs=n_cores)(delayed(run_FATS)(i, n) for i in batch)

		 [+] Total: 509
	  /Users/Carlos/Desktop/ExpI/Corot/RRab/EN2_STAR_CHR_0102812302.csv
	  /Users/Carlos/Desktop/ExpI/Corot/RRab/EN2_STAR_CHR_0102573994.csv
	  /Users/Carlos/Desktop/ExpI/Corot/RRab/EN2_STAR_CHR_0102824709.csv


/Users/Carlos/Desktop/ExpI/env/lib/python2.7/site-packages/scipy/optimize/_minimize.py:394: RuntimeWarning: Method nelder-mead cannot handle constraints nor bounds.
  RuntimeWarning)
/Users/Carlos/Desktop/ExpI/env/lib/python2.7/site-packages/scipy/optimize/_minimize.py:394: RuntimeWarning: Method nelder-mead cannot handle constraints nor bounds.
  RuntimeWarning)
/Users/Carlos/Desktop/ExpI/env/lib/python2.7/site-packages/scipy/optimize/_minimize.py:394: RuntimeWarning: Method nelder-mead cannot handle constraints nor bounds.
  RuntimeWarning)
/Users/Carlos/Desktop/ExpI/env/lib/python2.7/site-packages/FATS/FeatureFunctionLib.py:948: RuntimeWarning: invalid value encountered in power
  ((2 * np.pi * (Omega[i] + error_vars[i])) ** -0.5) *
/Users/Carlos/Desktop/ExpI/env/lib/python2.7/site-packages/FATS/FeatureFunctionLib.py:950: RuntimeWarning: divide by zero encountered in log
  (Omega[i] + error_vars[i]))) + epsilon))
/Users/Carlos/Desktop/ExpI/env/lib/python2.7/site-packages/FATS/Featur

	  /Users/Carlos/Desktop/ExpI/Corot/RRab/EN2_STAR_CHR_0102879782.csv
	  /Users/Carlos/Desktop/ExpI/Corot/RRab/EN2_STAR_CHR_0103800818.csv
	  /Users/Carlos/Desktop/ExpI/Corot/RRab/EN2_STAR_CHR_0104190253.csv
	  /Users/Carlos/Desktop/ExpI/Corot/RRab/EN2_STAR_CHR_0105173544.csv
	  /Users/Carlos/Desktop/ExpI/Corot/RRab/EN2_STAR_CHR_0105173544v2.csv
	  /Users/Carlos/Desktop/ExpI/Corot/RRab/EN2_STAR_CHR_0205924190.csv


KeyboardInterrupt: 

## Recover Files

In [32]:
dictionary = np.load('/Users/Carlos/Desktop/ExpI/Corot/RRab/Fats/500/EN2_STAR_CHR_0102879782.npy').item()